In [ ]:
import cv2
import numpy as np
import tkinter as tk
import pandas as pd
import json
import os
from tkinter import Label, Button, Text
from PIL import Image, ImageTk
from deepface import DeepFace

# Load Models
def load_models():
    proto_path_gender = "gender_deploy.prototxt"
    model_path_gender = "gender_net.caffemodel"
    proto_path_age = "age_deploy.prototxt"
    model_path_age = "age_net.caffemodel"

    try:
        gender_net = cv2.dnn.readNetFromCaffe(proto_path_gender, model_path_gender)
        age_net = cv2.dnn.readNetFromCaffe(proto_path_age, model_path_age)
    except cv2.error as e:
        print(f"❌ Error loading models: {e}")
        return None, None  

    if gender_net.empty() or age_net.empty():
        print("❌ One or more models failed to load. Check file paths!")
        return None, None  

    return gender_net, age_net

# Face detection and prediction
def detect_faces_and_predict(frame, gender_net, age_net):
    if gender_net is None or age_net is None:
        return []  

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return []  

    results = []
    age_ranges = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        # Ensure valid face before processing
        if face.shape[0] < 30 or face.shape[1] < 30:
            continue  # Skip small faces

        face_blob = cv2.dnn.blobFromImage(face, scalefactor=1.0, size=(227, 227), 
                                          mean=(78.426, 87.769, 114.896), 
                                          swapRB=False, crop=False)

        # Gender Prediction
        gender_net.setInput(face_blob)
        gender_preds = gender_net.forward()
        gender_confidence = max(gender_preds[0]) * 100  
        gender = "Male" if gender_preds[0][0] > gender_preds[0][1] else "Female"

        # Age Prediction
        age_net.setInput(face_blob)
        age_preds = age_net.forward()
        age_idx = np.argmax(age_preds[0])
        age_confidence = age_preds[0][age_idx] * 100  
        age = age_ranges[age_idx]

        # Emotion Prediction using DeepFace
        temp_path = "temp_face.jpg"
        cv2.imwrite(temp_path, face)
        
        emotion = "Unknown"
        emotion_confidence = 0
        
        try:
            analysis = DeepFace.analyze(temp_path, actions=["emotion"], enforce_detection=False)
            if analysis and isinstance(analysis, list):
                emotion = analysis[0]["dominant_emotion"]
                emotion_confidence = analysis[0]["emotion"].get(emotion, 0)
        except Exception as e:
            print(f"❌ DeepFace Emotion Detection Error: {e}")

        os.remove(temp_path)  # Clean up temp file

        results.append(((x, y, w, h), gender, gender_confidence, age, age_confidence, emotion, emotion_confidence))

    return results

# Update Frame in Real-Time
def update_frame():
    global cap, photo_label
    ret, frame = cap.read()
    if ret:
        results = detect_faces_and_predict(frame, gender_net, age_net)
        save_results(results)  # Save results in CSV/JSON
        
        for ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in results:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            text = f"{gender} ({gender_conf:.1f}%), {age} ({age_conf:.1f}%), {emotion} ({emotion_conf:.1f}%)"
            cv2.putText(frame, text, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = ImageTk.PhotoImage(img)
        photo_label.imgtk = img
        photo_label.configure(image=img)
    
    root.after(10, update_frame)

# Save results to CSV/JSON
def save_results(results):
    csv_filename = "detection_results.csv"
    json_filename = "detection_results.json"

    data = []
    for ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in results:
        data.append({
            "gender": gender,
            "gender_confidence": f"{gender_conf:.1f}%",
            "age": age,
            "age_confidence": f"{age_conf:.1f}%",
            "emotion": emotion,
            "emotion_confidence": f"{emotion_conf:.1f}%"
        })

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(csv_filename, mode='a', index=False, header=not os.path.exists(csv_filename))

    # Save to JSON
    with open(json_filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

# Detect and Show Results in GUI
def detect_and_show_results():
    global result_text, cap
    ret, frame = cap.read()
    if ret:
        results = detect_faces_and_predict(frame, gender_net, age_net)
        save_results(results)  # Save to file
        result_text.delete(1.0, tk.END)
        
        for idx, ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in enumerate(results):
            result_text.insert(tk.END, f"Person {idx+1}:\n")
            result_text.insert(tk.END, f"  Gender: {gender} ({gender_conf:.1f}%)\n")
            result_text.insert(tk.END, f"  Age Range: {age} ({age_conf:.1f}%)\n")
            result_text.insert(tk.END, f"  Emotion: {emotion} ({emotion_conf:.1f}%)\n\n")

# Load models
gender_net, age_net = load_models()

if gender_net is None or age_net is None:
    print("❌ Models failed to load. Exiting program.")
    exit()

# Setup GUI
root = tk.Tk()
root.title("Age, Gender & Emotion Detection with Confidence Scores")

photo_label = Label(root)
photo_label.pack()

btn_detect = Button(root, text="Detect", command=detect_and_show_results)
btn_detect.pack()

result_text = Text(root, height=10, width=60)
result_text.pack()

cap = cv2.VideoCapture(0)
update_frame()

root.mainloop()

# Release resources only after GUI closes
cap.release()
cv2.destroyAllWindows()


### The system supports both USB webcams and IP camera streams (such as an iPhone camera) and features a Tkinter-based GUI for ease of use.

In [ ]:
import cv2
import numpy as np
import tkinter as tk
import pandas as pd
import json
import os
from tkinter import Label, Button, Text
from PIL import Image, ImageTk
from deepface import DeepFace

# iPhone Stream Configuration
ip = "192.168.43.2"
port = "4747"
username = "admin"
password = "admin"
url = f"http://{username}:{password}@{ip}:{port}/video"

# Load Models
def load_models():
    proto_path_gender = "gender_deploy.prototxt"
    model_path_gender = "gender_net.caffemodel"
    proto_path_age = "age_deploy.prototxt"
    model_path_age = "age_net.caffemodel"

    try:
        gender_net = cv2.dnn.readNetFromCaffe(proto_path_gender, model_path_gender)
        age_net = cv2.dnn.readNetFromCaffe(proto_path_age, model_path_age)
    except cv2.error as e:
        print(f"❌ Error loading models: {e}")
        return None, None  

    if gender_net.empty() or age_net.empty():
        print("❌ One or more models failed to load. Check file paths!")
        return None, None  

    return gender_net, age_net

# Face detection and prediction
def detect_faces_and_predict(frame, gender_net, age_net):
    if gender_net is None or age_net is None:
        return []  

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return []  

    results = []
    age_ranges = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        # Ensure valid face before processing
        if face.shape[0] < 30 or face.shape[1] < 30:
            continue  # Skip small faces

        face_blob = cv2.dnn.blobFromImage(face, scalefactor=1.0, size=(227, 227), 
                                          mean=(78.426, 87.769, 114.896), 
                                          swapRB=False, crop=False)

        # Gender Prediction
        gender_net.setInput(face_blob)
        gender_preds = gender_net.forward()
        gender_confidence = max(gender_preds[0]) * 100  
        gender = "Male" if gender_preds[0][0] > gender_preds[0][1] else "Female"

        # Age Prediction
        age_net.setInput(face_blob)
        age_preds = age_net.forward()
        age_idx = np.argmax(age_preds[0])
        age_confidence = age_preds[0][age_idx] * 100  
        age = age_ranges[age_idx]

        # Emotion Prediction using DeepFace
        temp_path = "temp_face.jpg"
        cv2.imwrite(temp_path, face)
        
        emotion = "Unknown"
        emotion_confidence = 0
        
        try:
            analysis = DeepFace.analyze(temp_path, actions=["emotion"], enforce_detection=False)
            if analysis and isinstance(analysis, list):
                emotion = analysis[0]["dominant_emotion"]
                emotion_confidence = analysis[0]["emotion"].get(emotion, 0)
        except Exception as e:
            print(f"❌ DeepFace Emotion Detection Error: {e}")

        os.remove(temp_path)  # Clean up temp file

        results.append(((x, y, w, h), gender, gender_confidence, age, age_confidence, emotion, emotion_confidence))

    return results

# Update Frame in Real-Time
def update_frame():
    global cap, photo_label
    ret, frame = cap.read()
    if ret:
        results = detect_faces_and_predict(frame, gender_net, age_net)
        save_results(results)  # Save results in CSV/JSON
        
        for ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in results:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            text = f"{gender} ({gender_conf:.1f}%), {age} ({age_conf:.1f}%), {emotion} ({emotion_conf:.1f}%)"
            cv2.putText(frame, text, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = ImageTk.PhotoImage(img)
        photo_label.imgtk = img
        photo_label.configure(image=img)
    
    root.after(10, update_frame)

# Save results to CSV/JSON
def save_results(results):
    csv_filename = "detection_results.csv"
    json_filename = "detection_results.json"

    data = []
    for ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in results:
        data.append({
            "gender": gender,
            "gender_confidence": f"{gender_conf:.1f}%",
            "age": age,
            "age_confidence": f"{age_conf:.1f}%",
            "emotion": emotion,
            "emotion_confidence": f"{emotion_conf:.1f}%"
        })

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(csv_filename, mode='a', index=False, header=not os.path.exists(csv_filename))

    # Save to JSON
    with open(json_filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

# Detect and Show Results in GUI
def detect_and_show_results():
    global result_text, cap
    ret, frame = cap.read()
    if ret:
        results = detect_faces_and_predict(frame, gender_net, age_net)
        save_results(results)  # Save to file
        result_text.delete(1.0, tk.END)
        
        for idx, ((x, y, w, h), gender, gender_conf, age, age_conf, emotion, emotion_conf) in enumerate(results):
            result_text.insert(tk.END, f"Person {idx+1}:\n")
            result_text.insert(tk.END, f"  Gender: {gender} ({gender_conf:.1f}%)\n")
            result_text.insert(tk.END, f"  Age Range: {age} ({age_conf:.1f}%)\n")
            result_text.insert(tk.END, f"  Emotion: {emotion} ({emotion_conf:.1f}%)\n\n")

# Load models
gender_net, age_net = load_models()

if gender_net is None or age_net is None:
    print("❌ Models failed to load. Exiting program.")
    exit()

# Setup GUI
root = tk.Tk()
root.title("Age, Gender & Emotion Detection with Confidence Scores")

photo_label = Label(root)
photo_label.pack()

btn_detect = Button(root, text="Detect", command=detect_and_show_results)
btn_detect.pack()

result_text = Text(root, height=10, width=60)
result_text.pack()

# Connect to iPhone Video Stream
cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("❌ Failed to connect to iPhone video stream. Check IP, port, and app settings.")
    exit()

update_frame()

root.mainloop()

# Release resources only after GUI closes
cap.release()
cv2.destroyAllWindows()
